In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Define the fitness function (classification accuracy)
def fitness_function(features, X_train, y_train, X_test, y_test):
    # Subset the features: Convert the binary array into the actual feature subset
    X_train_subset = X_train[:, features.astype(bool)] if np.any(features) else X_train[:, :1]  # Default to 1 feature if no feature is selected
    X_test_subset = X_test[:, features.astype(bool)] if np.any(features) else X_test[:, :1]

    # Train a classifier (e.g., SVM)
    clf = SVC()
    clf.fit(X_train_subset, y_train)

    # Make predictions and calculate accuracy
    y_pred = clf.predict(X_test_subset)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Cuckoo Search Algorithm
def cuckoo_search(X, y, num_nests=10, max_iter=100, pa=0.25):
    n_features = X.shape[1]

    # Initialize nests (random binary feature subsets)
    nests = np.random.randint(2, size=(num_nests, n_features))

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Evaluate fitness of each nest
    fitness = np.array([fitness_function(nest, X_train, y_train, X_test, y_test) for nest in nests])

    # Best solution (initially the best nest)
    best_nest = nests[np.argmax(fitness)]
    best_fitness = np.max(fitness)

    # Main Cuckoo Search Loop
    for iteration in range(max_iter):
        # Generate new solutions using Levy flight
        new_nests = nests + np.random.normal(0, 1, nests.shape) * (best_nest - nests)
        new_nests = np.clip(new_nests, 0, 1)  # Ensure binary values (0 or 1)

        # Evaluate the fitness of the new nests
        new_fitness = np.array([fitness_function(nest, X_train, y_train, X_test, y_test) for nest in new_nests])

        # Select nests to replace (based on fitness)
        replace_mask = new_fitness > fitness
        nests[replace_mask] = new_nests[replace_mask]
        fitness[replace_mask] = new_fitness[replace_mask]

        # Abandon worst nests based on probability
        abandon_mask = np.random.rand(num_nests) < pa
        nests[abandon_mask] = np.random.randint(2, size=(np.sum(abandon_mask), n_features))

        # Update the best solution
        best_nest = nests[np.argmax(fitness)]
        best_fitness = np.max(fitness)

        # Print progress (optional)


    return best_nest, best_fitness

# Example usage
if __name__ == "__main__":
    # Load your dataset (e.g., Iris dataset)
    from sklearn.datasets import load_iris
    X, y = load_iris(return_X_y=True)

    # Run Cuckoo Search for feature selection
    best_features, best_accuracy = cuckoo_search(X, y, num_nests=10, max_iter=100)

    print("Best feature subset:", best_features)
    print("Best classification accuracy:", best_accuracy)


Best feature subset: [1 0 1 0]
Best classification accuracy: 1.0
